In [ ]:
# !pip install skater
# if you face an issue with this you can download skater master from here :https://github.com/oracle/Skater
# go that folder on your command prompt and run 
# python setup.py install
# also if you build skater like that , you'll need to restart your kernel for imports to start working

# for windows users if you get error : command not recognised 
# you need to add python to your environment variable path 

# follow this link to do the same : https://geek-university.com/python/add-python-to-the-windows-path/

# you will to have to also figure out where python is installed on your machine 


In [ ]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import warnings 
warnings.filterwarnings('ignore')
from mypipes_linear import *

In [ ]:
file=r'/Users/lalitsachan/Dropbox/0.0 Data/census_income.csv'

ci=pd.read_csv(file)
ci=ci.sample(1000)

In [ ]:
ci['Y'].unique()

In [ ]:
cat_vars=list(ci.select_dtypes(include=['object']).columns)

In [ ]:
cat_vars=[_ for _ in cat_vars if _ not in ['Y','education']]

In [ ]:
num_vars=list(ci.select_dtypes(exclude=['object']).columns)

In [ ]:
p1=pdPipeline([
    ('cat_select',VarSelector(cat_vars)),
    ('missing_trt',DataFrameImputer()),
    ('create_dummies',get_dummies_Pipe(100))
])

p2=pdPipeline([
    ('num_select',VarSelector(num_vars)),
    ('missing_trt',DataFrameImputer())
])

data_pipe=FeatureUnion([
    ('cat_pipe',p1),
    ('num_pipe',p2)
])

In [ ]:
data_pipe.fit(ci)

In [ ]:
x=pd.DataFrame(data=data_pipe.transform(ci),
                     columns=data_pipe.get_feature_names())
y=(ci['Y']==" >50K").astype(int)


In [ ]:
y.value_counts()

In [ ]:
rf=RandomForestClassifier(**{'criterion': 'entropy', 'max_depth': 14, 'max_features': 11, 'n_estimators': 320})

In [ ]:
rf.fit(x,y)

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

# if you get the error : ModuleNotFoundError: No module named 'sklearn.externals.six'

# do the following : go the file in your folder where the error is coming from 
# for me the issue was on top line of this file :

# ~/anaconda3/lib/python3.7/site-packages/skater-1.1.2b1-py3.7.egg/skater/core/visualizer/tree_visualizer.py
# change the line : from sklearn.externals.six import StringIO

# to : from six import StringIO
# save and restart the kernel 

# if you further get and error that no module named six then run :
# !pip install six

In [ ]:
interpreter = Interpretation(x, feature_names=x.columns)
model = InMemoryModel(rf.predict_proba, examples=x)
plots = interpreter.feature_importance.plot_feature_importance(model, ascending=True)


In [ ]:
pyint_model = InMemoryModel(rf.predict_proba, examples=x, target_names=['Y_0','Y_1'])
axes_list = interpreter.partial_dependence.plot_partial_dependence(['num_pipe__education.num'],
                                                                   pyint_model, 
                                                                   grid_resolution=30, 
                                                                   with_variance=True,
                                                                   figsize = (10, 5))


In [ ]:
axes_list = interpreter.partial_dependence.plot_partial_dependence(['num_pipe__age'],
                                                                   pyint_model, 
                                                                   grid_resolution=30, 
                                                                   with_variance=True,
                                                                   figsize = (10, 5))

In [ ]:
model = InMemoryModel(rf.predict_proba, examples=x, feature_names=x.columns,
                      target_names=['Y_0', 'Y_1'])
interpreter.partial_dependence.plot_partial_dependence([('num_pipe__age', 'num_pipe__education.num')], model, 
                                                       grid_resolution=10)

In [ ]:
from skater.util.dataops import show_in_notebook

In [ ]:
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer
from skater.util.dataops import show_in_notebook
exp = LimeTabularExplainer(x.values, 
                           feature_names=x.columns, 
                           discretize_continuous=True, 
                           class_names=['Y_0', 'Y_1'])

In [ ]:
exp.explain_instance(x.iloc[10].values, rf.predict_proba).show_in_notebook()

In [ ]:
surrogate_explainer = interpreter.tree_surrogate(oracle=model,max_depth=8)
y_pred_rf=rf.predict(x)

surrogate_explainer.fit(x, y_pred_rf, use_oracle=True, prune='post', scorer_type='default')

# if you get this error : TypeError: __init__() got an unexpected keyword argument 'presort'

# go to tree_surrogate.py file here : 
# ~/anaconda3/lib/python3.7/site-packages/skater-1.1.2b1-py3.7.egg/skater/core/global_interpretation/

# and remove the presort input to DecisionTreeClassfier/Regressor

In [ ]:
# if you get graphviz error on mac : please install by running this : sudo brew install graphviz
# if that gives error for brew not found , install homebrew
surrogate_explainer.plot_global_decisions(colors=['coral', 'lightsteelblue','darkkhaki'],
                                          file_name='simple_tree_pre.png')

In [ ]:
show_in_notebook('simple_tree_pre.png', width=400, height=300)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(rf.predict(x),y)

In [ ]:
accuracy_score(surrogate_explainer.predict(x),y)